In [1]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

import sys

# Append the path to Pydoop to sys.path
#sys.path.append("/usr/local/lib/python3.8/dist-packages")

Spark Configuration

In [2]:
# Spark Session - 2 CORE - imporvement CONFIGURATION!!
# spark_session = SparkSession.builder\
#     .master("local[2]")\
#     .appName("pseudo_spark_nora")\
#     .getOrCreate()

# Set Hadoop configuration directory
#os.environ['HADOOP_CONF_DIR'] = '/path/to/hadoop/conf'
#spark_session.stop()
#os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

spark_session = SparkSession.builder \
    .appName("Analyse_1000Files") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# num executors 
# executor memory RAM 


# spark_session = SparkSession.builder \
#     .appName("HDFS_Connection_Test_nora") \
#     .master("yarn") \
#     .config("spark.executor.instances", "2") \
#     .config("spark.hadoop.fs.default.name", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.defaultFS", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
#     .config("spark.hadoop.dfs.server.namenode.class", "org.apache.hadoop.hdfs.server.namenode.NameNode") \
#     .config("spark.hadoop.conf", "org.apache.hadoop.hdfs.HdfsConfiguration") \
#     .getOrCreate()

#.config("spark.dynamicAllocation.enabled", False)\
#.config("spark.cores.max", 4)\


#.config("spark.jars.packages", "LLNL:spark-hdf5:0.0.4") \
# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 03:21:00 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
/home/hadoop/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


File Path

In [3]:
#path = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI/TRAAAAW128F429D538.asci'
#path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset_ASCI_analysis_songs/'

# path_0 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_0/'
# path_1 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_1/'
# path_2 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_2/'
# path_3 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_3/'
# path_4 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_4/'
# path_5 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_5/'
# paths = [path_0, path_1, path_2, path_3, path_4, path_5]

path = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI/'

In [4]:
file_contents = spark_context.wholeTextFiles(path).map(lambda x: x[1].replace('\n', '').replace('{', '').replace('}', '').replace(' ', '').split(', '))
#file_contents = spark_context.wholeTextFiles(','.join(paths)).map(lambda x: x[1].replace('\n', '').replace('{', '').replace('}', '').replace(' ', '').split(', '))

24/03/15 03:21:18 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 343.6 KiB, free 366.0 MiB)
24/03/15 03:21:18 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 33.1 KiB, free 365.9 MiB)
24/03/15 03:21:18 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on master-node:36759 (size: 33.1 KiB, free: 366.3 MiB)
24/03/15 03:21:18 INFO SparkContext: Created broadcast 0 from wholeTextFiles at NativeMethodAccessorImpl.java:0


In [5]:
split_file_contents = file_contents.map(lambda x: x[0].split(','))

In [6]:
selected_elements = split_file_contents.map(lambda x: [float(x[i]) for i in [3, 4, 26, 23, 24, 25, 27, 28, 29]])

In [7]:
#selected_elements.first()

## Data Frame

In [8]:
schema = StructType([
    StructField("duration", FloatType(), True),
    StructField("end_of_fade_in", FloatType(), True),
    StructField("start_of_fade_out", FloatType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", FloatType(), True),
    StructField("mode_confidence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("time_signature_confidence", FloatType(), True)
])

In [9]:
df = spark_session.createDataFrame(selected_elements, schema).cache()

24/03/15 03:21:19 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/15 03:21:19 INFO SharedState: Warehouse path is 'file:/home/hadoop/MSD_Project/DataEngineeringProject/Scalability_Experiments/spark-warehouse'.
24/03/15 03:21:19 INFO ServerInfo: Adding filter to /SQL: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/15 03:21:19 INFO ServerInfo: Adding filter to /SQL/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/15 03:21:19 INFO ServerInfo: Adding filter to /SQL/execution: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/15 03:21:19 INFO ServerInfo: Adding filter to /SQL/execution/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/15 03:21:19 INFO ServerInfo: Adding filter to /static/sql: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter


In [10]:
num_rows = df.count()
print("Number of rows in the DataFrame:", num_rows)

24/03/15 03:21:22 INFO CodeGenerator: Code generated in 346.844432 ms
24/03/15 03:21:23 INFO FileInputFormat: Total input files to process : 10000
24/03/15 03:21:24 INFO FileInputFormat: Total input files to process : 10000
24/03/15 03:21:24 INFO DAGScheduler: Got job 0 (count at NativeMethodAccessorImpl.java:0) with 2 output partitions
24/03/15 03:21:24 INFO DAGScheduler: Final stage: ResultStage 0 (count at NativeMethodAccessorImpl.java:0)
24/03/15 03:21:24 INFO DAGScheduler: Parents of final stage: List()
24/03/15 03:21:24 INFO DAGScheduler: Missing parents: List()
24/03/15 03:21:24 INFO DAGScheduler: Submitting ResultStage 0 (*(1) Scan ExistingRDD[duration#0,end_of_fade_in#1,start_of_fade_out#2,loudness#3,mode#4,mode_confidence#5,tempo#6,time_signature#7,time_signature_confidence#8]
 MapPartitionsRDD[8] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/15 03:21:24 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 30.4 KiB,

Number of rows in the DataFrame: 10000


In [11]:
df.show()

24/03/15 03:21:35 INFO CodeGenerator: Code generated in 32.586267 ms
24/03/15 03:21:35 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/15 03:21:35 INFO DAGScheduler: Got job 3 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/15 03:21:35 INFO DAGScheduler: Final stage: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0)
24/03/15 03:21:35 INFO DAGScheduler: Parents of final stage: List()
24/03/15 03:21:35 INFO DAGScheduler: Missing parents: List()
24/03/15 03:21:35 INFO DAGScheduler: Submitting ResultStage 4 (MapPartitionsRDD[21] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/15 03:21:35 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 38.0 KiB, free 365.9 MiB)
24/03/15 03:21:35 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 15.3 KiB, free 365.9 MiB)
24/03/15 03:21:35 INFO BlockManagerInfo: Added broadcast_

+--------+--------------+-----------------+--------+----+---------------+-------+--------------+-------------------------+
|duration|end_of_fade_in|start_of_fade_out|loudness|mode|mode_confidence|  tempo|time_signature|time_signature_confidence|
+--------+--------------+-----------------+--------+----+---------------+-------+--------------+-------------------------+
| 218.932|         0.247|          218.932| -11.197| 0.0|          0.636| 92.198|           4.0|                    0.778|
| 148.035|         0.148|          137.915|  -9.843| 0.0|           0.43|121.274|           4.0|                    0.384|
| 177.475|         0.282|          172.304|  -9.689| 1.0|          0.565| 100.07|           1.0|                      0.0|
| 233.404|           0.0|          217.124|  -9.013| 1.0|          0.749|119.293|           4.0|                      0.0|
| 209.606|         0.066|          198.699|  -4.501| 1.0|          0.371|129.738|           4.0|                    0.562|
| 267.702|      

24/03/15 03:21:37 INFO CodeGenerator: Code generated in 30.818988 ms


# Calculation (Average)

In [12]:
average_values = df.agg({'duration': 'avg', 'end_of_fade_in': 'avg', 'start_of_fade_out': 'avg', 
                         'loudness': 'avg', 'mode': 'avg', 'mode_confidence': 'avg', 
                         'tempo': 'avg', 'time_signature': 'avg', 
                         'time_signature_confidence': 'avg'}).collect()[0]

24/03/15 03:21:37 INFO CodeGenerator: Code generated in 77.477609 ms
24/03/15 03:21:37 INFO DAGScheduler: Registering RDD 26 (collect at /tmp/ipykernel_302089/2441391651.py:1) as input to shuffle 1
24/03/15 03:21:37 INFO DAGScheduler: Got map stage job 4 (collect at /tmp/ipykernel_302089/2441391651.py:1) with 2 output partitions
24/03/15 03:21:37 INFO DAGScheduler: Final stage: ShuffleMapStage 5 (collect at /tmp/ipykernel_302089/2441391651.py:1)
24/03/15 03:21:37 INFO DAGScheduler: Parents of final stage: List()
24/03/15 03:21:37 INFO DAGScheduler: Missing parents: List()
24/03/15 03:21:37 INFO DAGScheduler: Submitting ShuffleMapStage 5 (MapPartitionsRDD[26] at collect at /tmp/ipykernel_302089/2441391651.py:1), which has no missing parents
24/03/15 03:21:37 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 59.6 KiB, free 365.8 MiB)
24/03/15 03:21:37 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 21.7 KiB, free 365.8 MiB

In [13]:
avg_df = pd.DataFrame([average_values.asDict()])

In [14]:
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 10000


In [15]:
avg_df

,avg(duration),avg(tempo),avg(end_of_fade_in),avg(start_of_fade_out),avg(time_signature),avg(time_signature_confidence),avg(mode_confidence),avg(mode),avg(loudness)
0,238.507519,122.915449,0.758616,229.975465,3.5648,0.509937,0.477784,0.6911,-10.485669


# STOP SPARK

In [16]:
# release the cores
spark_context.stop()

24/03/15 03:21:38 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/03/15 03:21:38 INFO SparkUI: Stopped Spark web UI at http://master-node:4040
24/03/15 03:21:38 INFO YarnClientSchedulerBackend: Interrupting monitor thread
24/03/15 03:21:38 INFO YarnClientSchedulerBackend: Shutting down all executors
24/03/15 03:21:38 INFO YarnSchedulerBackend$YarnDriverEndpoint: Asking each executor to shut down
24/03/15 03:21:38 INFO YarnClientSchedulerBackend: YARN client scheduler backend Stopped
24/03/15 03:21:38 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/03/15 03:21:38 INFO MemoryStore: MemoryStore cleared
24/03/15 03:21:38 INFO BlockManager: BlockManager stopped
24/03/15 03:21:38 INFO BlockManagerMaster: BlockManagerMaster stopped
24/03/15 03:21:38 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/03/15 03:21:38 INFO SparkContext: Successfully stopped SparkContext
